In [ ]:
from SelfCalGroupFinder.py.groupcatalog import GroupCatalog, TestGroupCatalog, serialize, deserialize
import SelfCalGroupFinder.py.catalog_definitions as cat
import SelfCalGroupFinder.py.pyutils
import SelfCalGroupFinder.py.plotting as pp
import numpy as np
import astropy.coordinates as coord
import sys
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Running Group Finder

This notebook will call the functions to preprocess, run group finder, and run postprocessing code to build up a GroupCatalog object, which mostly wraps a pandas DataFrame containing the resulting group catalog data.

After running this on a given GroupCatalog definition, a serialized (via pickle) version of the GroupCatalog object will exist which can be deserialized elsewhere for analysis. See post_plots.ipynb for that.

In [ ]:
# Run group finder on whatever datasets you want
datasets_to_run: list[GroupCatalog] = [
    #cat.sdss_vanilla,
    #cat.sdss_colors,
    #cat.sdss_colors_chi,
    #cat.mxxl_all,
    #cat.mxxl_all_c,
    #cat.mxxl_fiberonly,
    #cat.mxxl_fiberonly_c,
    #cat.mxxl_nn,
    #cat.mxxl_nn_c,
    #cat.mxxl_simple_2,
    #cat.mxxl_simple_2_c,
    #cat.mxxl_simple_4,
    #cat.mxxl_simple_4_c,
    #cat.uchuu_all,
    #cat.bgs_simple_4,
    #cat.bgs_simple_4_1pass,
    #cat.bgs_simple_4_no_sdss,
    #cat.bgs_simple_4_4p,
    #cat.bgs_simple_4_c,
    #cat.bgs_fiberonly,
    #cat.bgs_fiberonly_1pass,
    #cat.bgs_nn,
    #cat.bgs_nn_sdsslike,
    #cat.bgs_simple_2,
    #cat.bgs_simple_2_c,
    #cat.bgs_sv3_10p,
    #cat.bgs_sv3_10p_all,
]

# To just run postprocessing on GF output, comment out run_group_finder()
for d in datasets_to_run:
    #d = deserialize(d)
    d.run_group_finder(popmock=False)
    d.postprocess()
    #d.run_corrfunc()
    serialize(d)
    #del(d)


# Tests

In [ ]:
# Only needs to be run once, unless you want to change the test data
#catalog = TestGroupCatalog("Test")
#catalog.create_test_dat_files() 

In [ ]:
np.set_printoptions(threshold=sys.maxsize)

# Baseline vanilla group finder test 
catalog = TestGroupCatalog("Test")
catalog.run_group_finder(silent=True) 
catalog.postprocess()
df=catalog.all_data
baseline_total_mass = df['M_halo'].sum()
assert len(np.unique(df['igrp'])) == 200
assert len(df) == 246 
assert df['quiescent'].sum() == 129
assert np.isclose(df['weight'].sum(), 246 * 1.0) # no weights, just 1 per gal
m1=df['M_halo'].to_numpy()

# Test that when omega0 are 0, the others don't matter
catalog = TestGroupCatalog("Test")
catalog.GF_props['omegaL_sf'] = 123
catalog.GF_props['sigma_sf'] = 345
catalog.GF_props['omegaL_q'] = 456
catalog.GF_props['sigma_q'] = 678
catalog.GF_props['omega0_sf'] = 0.0
catalog.GF_props['omega0_q'] = 0.0
catalog.run_group_finder(silent=True)
catalog.postprocess()
df=catalog.all_data
assert len(np.unique(df['igrp'])) == 200
assert len(df) == 246 
assert df['quiescent'].sum() == 129
assert np.isclose(df['weight'].sum(), 246 * 1.0) # no weights, just 1 per gal
assert np.isclose(df['M_halo'].sum(), baseline_total_mass)
m2=df['M_halo'].to_numpy()

catalog = TestGroupCatalog("Test")
#catalog.GF_props['colors'] = 1
catalog.GF_props['omegaL_sf'] = 10.0
catalog.GF_props['sigma_sf'] = 3.0
catalog.GF_props['omegaL_q'] = 0.0
catalog.GF_props['sigma_q'] = 0.0
catalog.GF_props['omega0_sf'] = 10.0
catalog.GF_props['omega0_q'] = 0.0
catalog.run_group_finder(silent=True)
catalog.postprocess()
df=catalog.all_data
assert len(np.unique(df['igrp'])) >= 200 # these parameters make assigned halos smaller
assert len(df) == 246 
assert df['quiescent'].sum() == 129
assert df['weight'].sum() < 246 
# TODO BUG I feel like this should be true, but it's not. Weighting doesn't preseve the halo mass function
#assert np.isclose(df['M_halo'].sum(), baseline_total_mass) 
m3=df['M_halo'].to_numpy()

plt.hist(np.stack([np.log10(m1), np.log10(m2), np.log10(m3)], axis=-1))


print("All tests passed")

In [ ]:
pp.examine_area(np.min(df.RA), np.max(df.RA), np.min(df.Dec), np.max(df.Dec), df)
